In [80]:
import numpy as np
import pandas as pd

In [81]:
# Load data with specific id number
def load_data(id_num):
    data = np.load("eeg_data.npz")
    X = data['x']
    y = data['y']
    
    index = [i for i in range(len(y)) if y[i] == id_num]
    
    output_data = []
    output_label = []
    
    for i in index:
        output_data.append(X[i])
        output_label.append(y[i])
        
    return output_data, output_label

In [82]:
# Compare seeing one number with rest 
def binary_all_channel(data, label, id_num):
    if len(data) != len(label):
        print("Something is wrong here")
        return
    
    output_data = []
    output_label = []
    
    for i in range(len(label)):
        if label[i] != id_num and label[i] != -1:
            print("Something is wrong here")
            break
        if label[i] != -1:
            output_label.append(1)
        else:
            output_label.append(-1)
        
        feature = np.concatenate(data[i])
        feature = np.nan_to_num(feature)
        output_data.append(feature)
        
    return output_data, output_label    

In [83]:
def train(X, y):
    w = np.linalg.inv((X.transpose() @ X)) @ (X.transpose() @ y)
    return w

In [84]:
def binary_predict(X, w):
    raw_val = X.transpose() @ w
    
    if raw_val >= 0:
        return 1
    if raw_val < 0:
        return -1

In [85]:
def multiclass_predict(X, w):
    raw_val = X.transpose() @ w
    
    if raw_val >= 0:
        return 1
    if raw_val < 0:
        return -1

In [86]:
def cross_val(X, w):

SyntaxError: unexpected EOF while parsing (<ipython-input-86-7da3f0b7d18a>, line 1)

In [87]:
data_rest, label_rest = load_data(-1)
data_4, label_4 = binary_all_channel(data_rest, label_rest, 4)